In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kanchana1990/real-estate-data-london-2024")

print("Path to dataset files:", path)

c:\Users\topsn\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\topsn\.cache\kagglehub\datasets\kanchana1990\real-estate-data-london-2024\versions\1


In [2]:
import pandas as pd
df = pd.read_csv("realestate_data_london_2024_nov.csv")
df.describe(include='all')

,addedOn,title,descriptionHtml,propertyType,sizeSqFeetMax,bedrooms,bathrooms,listingUpdateReason,price
count,1011,1019,1019,1019,869.000000,1003.000000,984.000000,1019,1019
unique,442,965,1010,21,NaN,NaN,NaN,3,232
top,05/07/2024,"5 bedroom apartment for sale in Thames City, N...",The Residences at the Mandarin Oriental offers...,Apartment,NaN,NaN,NaN,new,"£6,500,000"
freq,15,4,3,243,NaN,NaN,NaN,702,57
mean,NaN,NaN,NaN,NaN,5232.871116,5.111665,4.648374,NaN,NaN
std,NaN,NaN,NaN,NaN,11796.770144,3.264992,3.085809,NaN,NaN
min,NaN,NaN,NaN,NaN,425.000000,1.000000,1.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,2885.000000,3.000000,3.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,3834.000000,5.000000,4.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,5745.000000,6.000000,5.000000,NaN,NaN


# Data Completeness

In [3]:
# Mising data
miss = df.isnull().sum()
print(miss)

addedOn                  8
title                    0
descriptionHtml          0
propertyType             0
sizeSqFeetMax          150
bedrooms                16
bathrooms               35
listingUpdateReason      0
price                    0
dtype: int64


## Missing value imputation
Will check down stream the performance of mode imputation on a model

In [4]:
df_mode_imputation = df.apply(lambda x: x.fillna(x.mode()[0]) if x.isnull().any() else x)
# Mising data
miss = df_mode_imputation.isnull().sum()
print(miss)

addedOn                0
title                  0
descriptionHtml        0
propertyType           0
sizeSqFeetMax          0
bedrooms               0
bathrooms              0
listingUpdateReason    0
price                  0
dtype: int64


## Sweet viz quick eda

In [9]:

from ydata_profiling import ProfileReport
profile = ProfileReport(df, title="Pandas Profiling Report")
df.profile_report()

Render HTML: 100%|██████████| 1/1 [00:00<00:00,  1.18it/s]


# EDA SUMMARY 
there is an issue with missing values and hgihly corelated features do to this I would avoid regression based models and porbably use a tree based algorithim such as random forest or xgboost
these algorithims also are robust to missing values and can handel them 

In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
# Assuming df is your DataFrame and the target column is 'target'

# 1. Data Preprocessing (Assuming 'target' is the label column)
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target variable

# Optional: Handle missing values (if any)
#X.fillna(X.mean(), inplace=True)

# Optional: Standardizing the features (important for models like XGBoost)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Train-Test Split (80-20 split)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# 3. Train XGBoost Model
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# 4. Predictions
y_pred = xgb_model.predict(X_test)
y_pred_prob = xgb_model.predict_proba(X_test)[:, 1]  # Probabilities for ROC-AUC

# 5. Performance Metrics
# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision:.4f}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall:.4f}')

# F1-Score
f1 = f1_score(y_test, y_pred)
print(f'F1-Score: {f1:.4f}')

# AUC-ROC Score
roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f'ROC-AUC Score: {roc_auc:.4f}')
  
# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print(f'Confusion Matrix:\n{conf_matrix}')


ValueError: could not convert string to float: '10/10/2024'